This notebook takes a look at the results from a hypothetical slot machine through the lens of probability
* Tutors:
  * Anthropic's AI Claude
  * Google's Gemini

Given a slot machine and five observed payouts

Observed numbers = [0,0,1.5,0,1.5]
  * Sum = 3
  * Average = 3/5 = 0.6
  * Sample size = 5



Methods used for developing probability estimates
* Guess 1 = discrete probability distribution, binomal distribution
* Guess 2 = continous probability distribution, normal distribution
* Guess 3 = monte carlo approach
* Guess 4 = bayseian approach

In [1]:
"""
Slot Machine Payout Analysis
---------------------------
This script analyzes slot machine payout patterns using multiple probability approaches:
1. Binomial Distribution (discrete probability)
2. Normal Distribution (continuous probability)
3. Monte Carlo Simulation
4. Bayesian Analysis

The analysis is based on observed payout data and provides various statistical measures
and probability estimates for future payouts.

Main Features:
- Basic statistical calculations (mean, median, standard deviation)
- Probability distribution analysis
- Monte Carlo simulation
- Bayesian probability estimation
- Comparative analysis of different approaches

Required Libraries:
- numpy
- scipy.stats
"""

import numpy as np
from scipy.stats import binom
from scipy import stats

# Basic Statistical Functions
class StatisticalCalculator:
    """
    A class containing basic statistical calculation methods.
    All calculations are implemented manually for educational purposes.
    """

    @staticmethod
    def calc_mean(numbers):
        """
        Calculate the arithmetic mean of a sequence of numbers.

        Args:
            numbers (list): List of numerical values

        Returns:
            float: Arithmetic mean
        """
        total = sum(numbers)
        pop = len(numbers)
        return total/pop

    @staticmethod
    def calc_median(numbers):
        """
        Calculate the median value of a sequence of numbers.

        Args:
            numbers (list): List of numerical values

        Returns:
            float: Median value
        """
        sorted_numbers = sorted(numbers)
        n = len(sorted_numbers)

        if n % 2 == 0:
            return (sorted_numbers[n//2 - 1] + sorted_numbers[n//2]) / 2
        else:
            return sorted_numbers[n//2]

    @staticmethod
    def calc_standard_deviation(numbers):
        """
        Calculate the population standard deviation.

        Args:
            numbers (list): List of numerical values

        Returns:
            float: Population standard deviation
        """
        n = len(numbers)
        mean = sum(numbers) / n
        squared_diff_sum = sum((x - mean) ** 2 for x in numbers)
        variance = squared_diff_sum / n
        return variance ** 0.5

# Probability Distribution Functions
class ProbabilityAnalyzer:
    """
    A class containing methods for different probability distribution analyses.
    """

    @staticmethod
    def binomial_distribution(k, n, p):
        """
        Calculate binomial probability mass function.

        Args:
            k (int): Number of successes
            n (int): Number of trials
            p (float): Probability of success on each trial

        Returns:
            float: Probability of exactly k successes in n trials
        """
        return binom.pmf(k, n, p)

    @staticmethod
    def normal_distribution(x, mu, sigma):
        """
        Calculate normal distribution probability density.

        Args:
            x (float): Point at which to evaluate
            mu (float): Mean of the distribution
            sigma (float): Standard deviation

        Returns:
            float: Probability density at point x
        """
        return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

    @staticmethod
    def monte_carlo_simulation(observed_data, n_simulations=10000, n_spins=5):
        """
        Perform Monte Carlo simulation for slot machine outcomes.

        Args:
            observed_data (list): Observed payout data
            n_simulations (int): Number of simulations to run
            n_spins (int): Number of spins per simulation

        Returns:
            dict: Simulation results including averages and distributions
        """
        unique_payouts = np.unique(observed_data)
        payout_probs = [np.mean(np.array(observed_data) == payout) for payout in unique_payouts]

        simulation_results = []
        win_counts = []

        for _ in range(n_simulations):
            spins = np.random.choice(unique_payouts, size=n_spins, p=payout_probs)
            simulation_results.append(np.mean(spins))
            win_counts.append(np.sum(spins > 0) / n_spins)

        return {
            'average_payout': np.mean(simulation_results),
            'win_probability': np.mean(win_counts),
            'confidence_interval': stats.norm.interval(0.95, loc=np.mean(win_counts),
                                                    scale=stats.sem(win_counts)),
            'payout_distribution': {float(payout): len([x for x in simulation_results
                                                      if x == payout]) / n_simulations
                                  for payout in unique_payouts}
        }

    @staticmethod
    def bayesian_slot_probability(observed_data, prior_prob=0.4):
        """
        Calculate Bayesian probability estimate for winning.

        Args:
            observed_data (list): Observed payout data
            prior_prob (float): Prior probability of winning

        Returns:
            float: Posterior probability of winning
        """
        wins = sum(1 for payout in observed_data if payout > 0)
        total_spins = len(observed_data)

        alpha = 1 + wins
        beta = 1 + (total_spins - wins)

        return alpha / (alpha + beta)

# Results Formatter
class ResultsFormatter:
    """
    A class to format and display analysis results.
    """

    @staticmethod
    def print_initial_analysis(numbers):
        """Print initial data analysis results."""
        stats = StatisticalCalculator()
        total_payout = sum(numbers)
        average_payout = total_payout / len(numbers)

        print("\nInitial Data Analysis:")
        print(f"Observed results: {numbers}")
        print(f"Sample size: {len(numbers)}")
        print(f"Observed wins: {sum(1 for x in numbers if x > 0)} out of {len(numbers)} spins")
        print(f"Average Payout: ${average_payout:.3f}\n")

        print(f"Mean: {stats.calc_mean(numbers)}")
        print(f"Median: {stats.calc_median(numbers)}")
        print(f"Standard deviation: {stats.calc_standard_deviation(numbers)}\n")

    @staticmethod
    def print_all_results(numbers):
        """Print comprehensive analysis results."""
        analyzer = ProbabilityAnalyzer()

        # Calculate all results first
        k, n, p = 2, 5, 0.4
        bin_prob = analyzer.binomial_distribution(k, n, p)
        expected_payout = p * 1.5

        mean = StatisticalCalculator.calc_mean(numbers)
        median = StatisticalCalculator.calc_median(numbers)
        std_dev = StatisticalCalculator.calc_standard_deviation(numbers)
        norm_prob = analyzer.normal_distribution(mean, median, std_dev)

        monte_carlo_results = analyzer.monte_carlo_simulation(numbers)
        posterior_prob = analyzer.bayesian_slot_probability(numbers)

        # Print all results
        ResultsFormatter.print_initial_analysis(numbers)

        # Print distribution results
        print("Binomial Distribution Results:")
        print(f"Average Payout: ${expected_payout:.3f}")
        print(f"Probability of exactly {k} wins in {n} spins: {bin_prob:.3f}")
        print(f"Parameters: n={n} trials, p={p:.2f} success probability\n")

        print("Normal Distribution Results:")
        print(f"Average Payout: ${median:.3f}")
        print(f"Probability density at mean: {norm_prob:.3f}")
        print(f"Distribution parameters: μ={median:.2f}, σ={std_dev:.2f}\n")

        print("Monte Carlo Simulation Results:")
        print(f"Average Payout: ${monte_carlo_results['average_payout']:.3f}")
        print(f"Win Probability: {monte_carlo_results['win_probability']:.3f}\n")

        print("Bayesian Analysis Results:")
        print(f"Average Payout: ${posterior_prob * 1.5:.3f}")
        print(f"Posterior probability of winning: {posterior_prob:.3f}\n")

        # Print summary
        print("Summary of Average Payouts Across Methods:")
        print(f"Observed Data Average:     ${sum(numbers)/len(numbers):.3f}")
        print(f"Binomial Expected:         ${expected_payout:.3f}")
        print(f"Normal Distribution Mean:  ${median:.3f}")
        print(f"Monte Carlo Simulated:     ${monte_carlo_results['average_payout']:.3f}")
        print(f"Bayesian Expected:         ${posterior_prob * 1.5:.3f}")

def main():
    """Main execution function."""
    # Sample data
    numbers = [0, 0, 1.5, 0, 1.5]

    # Print all results
    ResultsFormatter.print_all_results(numbers)

if __name__ == "__main__":
    main()


Initial Data Analysis:
Observed results: [0, 0, 1.5, 0, 1.5]
Sample size: 5
Observed wins: 2 out of 5 spins
Average Payout: $0.600

Mean: 0.6
Median: 0
Standard deviation: 0.7348469228349535

Binomial Distribution Results:
Average Payout: $0.600
Probability of exactly 2 wins in 5 spins: 0.346
Parameters: n=5 trials, p=0.40 success probability

Normal Distribution Results:
Average Payout: $0.000
Probability density at mean: 0.389
Distribution parameters: μ=0.00, σ=0.73

Monte Carlo Simulation Results:
Average Payout: $0.594
Win Probability: 0.396

Bayesian Analysis Results:
Average Payout: $0.643
Posterior probability of winning: 0.429

Summary of Average Payouts Across Methods:
Observed Data Average:     $0.600
Binomial Expected:         $0.600
Normal Distribution Mean:  $0.000
Monte Carlo Simulated:     $0.594
Bayesian Expected:         $0.643
